## Библиотеки

In [7]:
from sshtunnel import SSHTunnelForwarder
from pymongo import MongoClient
import json
from start import start_parser
import pprint
import openpyxl as px # работа с  xlsx файлами
import numpy

## Подключение к бд

In [8]:
MONGO_PORT = 55556
MONGO_DB = "fincase"
MONGO_USER = "root"
MONGO_PASS = "Gfcbyu&478532@#Gfhcbyu"
SERVER_HOST = '185.98.83.65'
SERVER_PORT = 20032

MONGO_PORT = 55556
MONGO_DB = "fincase"
MONGO_USER = "shesterin"
MONGO_PASS = "sK349Acm9qwD92s"
SERVER_HOST = '192.168.0.78'
SERVER_PORT = 22

In [9]:
server = SSHTunnelForwarder(
    (SERVER_HOST, 22),
    ssh_username = MONGO_USER,
    ssh_password = MONGO_PASS,
    remote_bind_address = ('localhost', MONGO_PORT)
)

server.start()
print(server.local_bind_port)

41339


In [12]:
client = MongoClient('localhost', server.local_bind_port)
db = client[MONGO_DB]
print (client.database_names())
print (client[MONGO_DB].collection_names())

['admin', 'config', 'fincase', 'local', 'test']
['storage', 'garage', 'building', 'carservice', 'psn', 'tradingArea', 'production', 'house', 'investmoscowLots', 'room', 'flat', 'land', 'office']


In [5]:
d = dict((db, [collection for collection in client[db].collection_names()])
        for db in client.database_names())
print (json.dumps(d))

{"admin": ["system.version"], "config": ["system.sessions"], "fincase": ["building", "storage", "garage", "psn", "tradingArea", "production", "room", "investmoscowLots", "flat", "land", "office"], "local": ["startup_log"]}


## Основная функция

In [7]:
def get_urls(client, MONGO_DB):
    try:
        for collection in db.collection_names():
            print (collection + ' start!')
            if collection == "building":
        # смотрим коллекцию объявлений где признак null, в нашем случае лучше сделать рекурсию,
        #что бы этот процесс не заканчивался
                for a in db[collection].find({"$and":
                                                [{"pageId": {"$ne": None}}, 
                                                        {"sematic":{"$eq": None}}]}, 
                                                           {"building_house_line_type": 1, 
                                                            "condition_type": 1, 
                                                            "description": 1,
                                                            "vat_type" : 1, 
                                                            "pageId": 1}).batch_size(30):
                    data = a['description']
                    result = start_parser(data)   
                    if a['building_house_line_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                    {"$set": 
                                                                     {"building_house_line_type": result.iloc[0]["houselinetype"]}})
                    if a['condition_type'] == None:
                            #a['condition_type'] = result.iloc[0]["conditiontype"] 
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                    {"$set": 
                                                                    {"condition_type": result.iloc[0]["conditiontype"]}})
                    if a['vat_type'] == None:
                            #a['vat_type'] = result.iloc[0]["vattype"] 
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                    {"$set": 
                                                                     {"vat_type": result.iloc[0]["vattype"]}})           
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                    {"$set": 
                                                                     {"sematic": "true"}})

            elif collection == "storage":
                for a in db[collection].find({"$and":
                                                            [{"pageId": {"$ne": None}}, 
                                                             {"sematic":{"$eq": None}}]}, 
                                                           {"floor_number": 1, 
                                                            "condition_type": 1,
                                                            "description": 1,
                                                            "vat_type" : 1, 
                                                            "pageId": 1}).batch_size(30):
                    data = a['description']
                    result = start_parser(data)
                    if a['floor_number'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"floor_number": result.iloc[0]["conditiontype"]}})
                    if a['condition_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"condition_type": result.iloc[0]["conditiontype"]}})
                    if a['vat_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"vat_type": result.iloc[0]["vattype"]}})
                    db[collection].update_one({"pageId": a["pageId"]},
                                                            {"$set": 
                                                            {"sematic": "true"}})
                    
            elif collection == "garage":
                for a in db[collection].find({"$and":
                                                            [{"pageId": {"$ne": None}}, 
                                                             {"sematic":{"$eq": None}}]}, 
                                                           {"vat_type" : 1,
                                                            "description": 1,
                                                            "pageId": 1}).batch_size(30):
                    data = a['description']
                    result = start_parser(data)       
                    if a['vat_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"vat_type": result.iloc[0]["vattype"]}})
                    db[collection].update_one({"pageId": a["pageId"]},
                                                            {"$set": 
                                                            {"sematic": "true"}})
                    
            elif collection == "psn":
                for a in db[collection].find({"$and":
                                                            [{"pageId": {"$ne": None}}, 
                                                             {"sematic":{"$eq": None}}]}, 
                                                           {"has_shop_windows": 1, 
                                                            "floor_number": 1, 
                                                            "condition_type": 1, 
                                                            "description": 1,
                                                            "vat_type" : 1, 
                                                            "pageId": 1}).batch_size(30):     
                    data = a['description']
                    result = start_parser(data)
                    if a['floor_number'] == None:
                        sd = result.iloc[0]["floornumber"]
                        print(type(sd))
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"floor_number": result.iloc[0]["floornumber"]}})                                 
                    if a['condition_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"condition_type": result.iloc[0]["conditiontype"]}})
                    if a['has_shop_windows'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"has_shop_windows": result.iloc[0]["hasshopwindows"]}}) 
                    if a['vat_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"vat_type": result.iloc[0]["vattype"]}})
                    db[collection].update_one({"pageId": a["pageId"]},
                                                            {"$set": 
                                                            {"sematic": "true"}})
                    
            elif collection == "tradingArea":
                for a in db[collection].find({"$and":
                                                            [{"pageId": {"$ne": None}}, 
                                                             {"sematic":{"$eq": None}}]}, 
                                                           {"floor_number": 1, 
                                                            "description": 1,
                                                            "condition_type": 1, 
                                                            "vat_type" : 1, 
                                                            "pageId": 1}).batch_size(30):
                    data = a['description']
                    result = start_parser(data)                         
                    if a['floor_number'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"floor_number": result.iloc[0]["floornumber"]}})                                  
                    if a['condition_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"condition_type": result.iloc[0]["conditiontype"]}})
                    if a['vat_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"vat_type": result.iloc[0]["vattype"]}})
                    db[collection].update_one({"pageId": a["pageId"]},
                                                            {"$set": 
                                                            {"sematic": "true"}})
                    
            elif collection == "production":
                for a in db[collection].find({"$and":
                                                            [{"pageId": {"$ne": None}}, 
                                                             {"sematic":{"$eq": None}}]}, 
                                                           {"floor_number": 1, 
                                                            "condition_type": 1, 
                                                            "description": 1,
                                                            "vat_type" : 1, 
                                                            "pageId": 1}).batch_size(30):
                    data = a['description']
                    result = start_parser(data)    
                    if a['floor_number'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"floor_number": result.iloc[0]["floornumber"]}})                                
                    if a['condition_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"condition_type": result.iloc[0]["conditiontype"]}})
                    if a['vat_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"vat_type": result.iloc[0]["vattype"]}})  
                    db[collection].update_one({"pageId": a["pageId"]},
                                                            {"$set": 
                                                            {"sematic": "true"}})       
            elif collection == "land":
                for a in db[collection].find({"$and":
                                                            [{"pageId": {"$ne": None}}, 
                                                             {"sematic":{"$eq": None}}]}, 
                                                             {"vat_type" : 1, 
                                                              "description": 1,
                                                              "pageId": 1}).batch_size(30):
                    data = a['description']
                    result = start_parser(data)       
                    if a['vat_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"vat_type": result.iloc[0]["vattype"]}})
                    db[collection].update_one({"pageId": a["pageId"]},
                                                            {"$set": 
                                                            {"sematic": "true"}})                  
            elif collection == "office":
                for a in db[collection].find({"$and":
                                                            [{"pageId": {"$ne": None}}, 
                                                             {"sematic":{"$eq": None}}]}, 
                                                             {"floor_number": 1, 
                                                              "condition_type": 1,
                                                              "description": 1,
                                                              "vat_type": 1,
                                                              "pageId": 1}).batch_size(30):
                    data = a['description']
                    result = start_parser(data) 
                    if a['floor_number'] == None:
                        res = result.iloc[0]["floornumber"]
                        if res == 0 or res ==-1 or res == 1 or res == 2:
                            db[collection].update_one({"pageId": a["pageId"]},
                                                                    {"$set": 
                                                                     {"floor_number": float(res)}})
                        elif res == "unknown":
                            db[collection].update_one({"pageId": a["pageId"]},
                                                                    {"$set": 
                                                                     {"floor_number": None}})

                        else:
                            db[collection].update_one({"pageId": a["pageId"]},
                                                                    {"$set": 
                                                                     {"floor_number": 999}})

                    if a['condition_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"condition_type": result.iloc[0]["conditiontype"]}})
                    if a['vat_type'] == None:
                        db[collection].update_one({"pageId": a["pageId"]},
                                                                {"$set": 
                                                                 {"vat_type": result.iloc[0]["vattype"]}})
                    db[collection].update_one({"pageId": a["pageId"]},
                                                            {"$set": 
                                                            {"sematic": "true"}})
            print (collection + ' done!')   
        #get_urls()   
            #print("1")
    except Exception as e:
        print(e)
def save_sematic_info(collection, id):
    collection.update_one({"pageId": id}, {"$set": {" sematic": 'true'}})

## Запуск парсера

In [8]:
get_urls(client, MONGO_DB)

storage start!
storage done!
garage start!
garage done!
building start!


KeyboardInterrupt: 

## Анализ результатов

In [44]:
array_building = [0]*5
array_storage = [0]*5
array_garage = [0]*5
array_psn = [0]*5
array_tradingArea = [0]*5
array_production = [0]*5
array_room = [0]*5
array_flat = [0]*5
array_land = [0]*1
array_flat = [0]*5
array_office = [0]*4

In [47]:
for a in client[MONGO_DB]["land"].find():    
    if a['vat_type'] == "unknown":
        array_land[0] += 1
        
for a in client[MONGO_DB]["office"].find():
    
    if a["floor_number"] == "unknown":
        array_office[0] += 1
                        
    if a['condition_type'] == "unknown":
        array_office[1] += 1
        
    if a['vat_type'] == "unknown":
        array_office[2] += 1
    
    if a["floor_number"] == 999:
        array_office[3] += 1    

## Выгрузка в Exel

In [50]:
tables = ["building", "storage", "garage", "psn", "tradingArea", "production", "room", "flat", "land", "office"]
arrays = [array_building, array_storage, array_garage,array_psn, array_tradingArea,
          array_production, array_room, array_flat, array_land, array_flat, array_office]
file = 'fill_db.xlsx'
work_sheet = u'Лист1'
wb = px.load_workbook(file, data_only=True)
sheet = wb[work_sheet]
j=0
for name in tables: 
    j+=1
    sheet.cell(row = j, column = 1).value = name
    sheet.cell(row = j, column = 3).value = str(arrays[j])
wb.save("fill_db.xlsx") 

In [43]:
file = 'table_columns1.xlsx'
work_sheet = u'Лист2'
wb = px.load_workbook(file, data_only=True)
sheet = wb[work_sheet]
(i, j) = (0, 1)
for collection in client[MONGO_DB].collection_names():
    i += 1
    sheet.cell(row = 1, column = i).value = str(collection)   
    j = 1
    string = client[MONGO_DB][collection].find_one()    
    for key in string.keys():
        j+=1
        sheet.cell(row = j, column = i).value = str(key)
        #print (key)
wb.save("table_columns1.xlsx") 

2019-10-11 11:31:06,030| ERROR   | Could not establish connection from ('127.0.0.1', 41603) to remote side of the tunnel
2019-10-11 11:42:15,179| ERROR   | Socket exception: Connection timed out (110)


In [9]:
client.close()

In [10]:
server.stop()

In [22]:
collection = "psn"
i=0
for a in db[collection].find({"$and":
                              [{"pageId": {"$ne": None}}, 
                               {"sematic":{"$eq": None}}
                              ]
                             }).batch_size(30): 
    i+=1
print(i)


12608
